## Evaluation of Manipulation experiments

In [ ]:
import os
import re
import numpy as np
import signal_logger
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

import ipywidgets as widgets 
from IPython.display import display

from path_extender import *
from utilities import *

%matplotlib notebook

In [ ]:
# to get plots of zbf comparison with alpha = 1000 and alpha = 1:
# ROOT_DIR = "/media/giuseppe/My Passport/Work/logs_mppi/passivity_zbf_comparison/"

# to get plots of passivity comparison (alpha = 1000) filter_in_out vs no_filter:
#ROOT_DIR = "/media/giuseppe/My Passport/Work/logs_mppi/with_without_passivity_object_stuck/"

# to get plots of all methods comparisons:
ROOT_DIR = "/media/giuseppe/My Passport/Work/logs_mppi/objects_interaction_method_analysis/"

LOG_PREFIX = ""
LOG_FILES = get_files(ROOT_DIR, LOG_PREFIX)
REQUIRED_FIELDS = [
    "log/sim_time", 
    "log/solver/rate",
    "log/solver/delay_steps",
    "log/solver/rollouts/min_cost",
    
    "log/opt_time",
    "log/simulation_step",
    "log/power_from_interaction",
    "log/stage_cost", 
    "log/torque_command",
    
    "log/external_torque",
    "log/ground_truth_external_torque",
    "log/external_wrench",
    "log/external_wrench_filtered",
    "log/position_desired",
    "log/position_measured",
    
    "log/velocity_command",
    "log/velocity_measured",
    "log/velocity_mppi",
    
    "log/cartesian_limits_violation", 
    "log/joint_limits_violation",
    "log/tank_state", 
    "log/power_channels",
    "log/state"
]
data = get_data(ROOT_DIR, REQUIRED_FIELDS, log_prefix=LOG_PREFIX)
data = process_data(data, final_time=40)
data = pd.DataFrame.from_dict(data)

<a id='index'></a>

### Index
- [Methods evaluation](#methods_evaluation)
- [Power cost analysis](#power_cost_analysis)
- [Energy tank comparison](#energy_tank_comparison)
- [Passivity coefficient analysis](#passivity_coefficient_analysis)
- [Experiment plots](#experiment_plots)

In [ ]:
DO_METHOD_EVALUATION = True
DO_EXPERIMENT_PLOTS = False
DO_POWER_COST_ANALYSIS = False
DO_PASSIVITY_COEFFICIENT_ANALYSIS = False 
DO_ENERGY_TANK_COMPARISON = True

<a id='methods_evaluation'></a>

## Method evaluation 
[index](#index)

### Stage cost

In [ ]:
if DO_METHOD_EVALUATION:
    order = ["no_filter", "filter_out", "filter_in", "filter_in_out"]

    g =sns.catplot(x="x", y="average_stage_cost", hue="experiment_type", col="object_type", 
                   hue_order=order, data=data, kind="bar",  capsize=.05, errwidth=5, edgecolor=".2", 
                   height=4, aspect=.7, lw=5,legend=False)

    (g.set_axis_labels("", "Average stage cost")
      .set_xticklabels([])
      .set_titles("{col_name}")
      .despine(right=False, top=False))

    #set_facetgrid_style(g, title=True, legend=True)
    display_save_button(g.fig)
 

### Joint limits
We compute the cumulative violation summed over all joints

In [ ]:
if DO_METHOD_EVALUATION:
    g =sns.catplot(x="x", y="joint_limits_violation_se", hue="experiment_type", col="object_type", 
                   hue_order=order, data=data, kind="bar", height=4, aspect=.7,
                   lw=5,  capsize=.05, errwidth=5, edgecolor=".2", legend=False);

    g.set_axis_labels("", "Joint limits violation")
    g.set_xticklabels([""])
    g.set_titles("{col_name}")
    g.despine(right=False, top=False)

    #set_facetgrid_style(g, title=True, legend=False)
    display_save_button(g.fig)

### Cartesian limits
We compute the cumulative squared violation of cartesian limits (self collision)

In [ ]:
if DO_METHOD_EVALUATION:
    g =sns.catplot(x="x", y="cartesian_limits_violation_se", hue="experiment_type", col="object_type", 
                   hue_order=order, data=data, kind="bar", height=4, aspect=.7,
                   lw=5,  capsize=.05, errwidth=5, edgecolor=".2", legend=False);

    g.set_axis_labels("", "Self collision violation")
    g.set_xticklabels([""])
    g.set_titles("{col_name}")
    g.despine(right=False, top=False)
    for _, ax in g.axes_dict.items():
        ax.set_ylim([0.0, 0.30])

    #set_facetgrid_style(g, title=True, legend=False)
    display_save_button(g.fig)

### Average wrench
We analyse the effect of the energy tank indirectly thorugh a minimized applied wrench

In [ ]:
if DO_METHOD_EVALUATION:
    g =sns.catplot(x="x", y="dissipated_power", hue="experiment_type", col="object_type", 
               hue_order=order, data=data, kind="bar", height=4, aspect=.7,
               lw=5,  capsize=.05, errwidth=5, edgecolor=".2", legend=False);

    g.set_axis_labels("", "Dissipated Power")
    g.set_xticklabels([""])
    g.set_titles("{col_name}")
    g.despine(right=False, top=False)

    #set_facetgrid_style(g, title=True, legend=False)
    display_save_button(g.fig)

<a id='power_cost_analysis'></a>

## Power vs No-Power cost
[index](#index)

We compare experiments where we do/dont penalize the power exchanged with the environment

In [ ]:
if DO_POWER_COST_ANALYSIS:
    # concatenate all values for each experiment
    all_wrench_data = {
        'power cost': [], 
        'object_type': [], 
        'wrench_norm': [], 
        'power': []
    }

    for power_cost, object_type, wrench, power in zip(data['power_cost'], 
                                                      data['object_type'], 
                                                      data['wrench_norm'], 
                                                      data['power_from_interaction']):
        # threshold wrench
        wrench_thresholded = wrench[wrench > 1]
        all_wrench_data['power cost'].extend(["On" if power_cost else "Off"] * len(wrench_thresholded))
        all_wrench_data['object_type'].extend([object_type] * len(wrench_thresholded))
        all_wrench_data['wrench_norm'].extend(wrench_thresholded)
        all_wrench_data['power'].extend(power[wrench>1])

    all_wrench_df = pd.DataFrame.from_dict(all_wrench_data)

    fig, ax = plt.subplots(2, 1)
    sns.barplot(x='object_type', y="power", hue="power cost", data=all_wrench_df, ax=ax[0],
                hue_order=["Off", "On"], lw=1.5,  capsize=.05, errwidth=2, edgecolor=".2",)

    sns.barplot(x='object_type', y="wrench_norm", hue="power cost", data=all_wrench_df, ax=ax[1],
                hue_order=["Off", "On"], lw=1.5,  capsize=.05, errwidth=2, edgecolor=".2",)

    ax[0].set_xlabel("")
    ax[0].set_ylabel("Power [W]",fontsize=18)
    ax[0].set_xticks([])
    ax[0].set_ylim([-25, 2])
    ax[0].tick_params(labelsize=15)

    ax[1].set_xlabel("")
    ax[1].set_ylabel("Wrench [N]",fontsize=18)
    ax[1].tick_params(labelsize=15)
    ax[1].legend().set_visible(False)


    display_save_button(fig)

<a id='energy_tank_comparison'></a>

## Wrench and energy tank
[index](#index)

In [ ]:
if DO_ENERGY_TANK_COMPARISON:
    # we need to align time across all the experiments 
    from scipy.interpolate import interp1d

    # find common min and max time as well as best approx for first fix and release time for the object
    t_min = 0
    t_max = np.inf
    t_min_fix = np.inf
    t_max_rel = 0
    TOLL = 1
    for t, s in zip(data['sim_time'], data['state']):
        if t[0] > t_min:
            t_min = t[0]
        if t[-1] < t_max:
            t_max = t[-1]

        # find where the object is released
        object_positions = s[:, 24]
        object_fix = np.where(object_positions > np.pi * (30 - TOLL) / 180)[0][0]
        object_release = np.where(object_positions > np.pi * (30 + TOLL) / 180)[0][0]

        fix_time = t[object_fix]
        release_time = t[object_release]
        if fix_time < t_min_fix:
            t_min_fix = fix_time
        if release_time > t_max_rel:
            t_max_rel = release_time

    global_time = np.arange(t_min, t_max, step=0.01)


    # concatenate data for each experiment in a single dictionary
    data_extended = {
        'time': [], 
        'wrench_norm': [], 
        'experiment_type': [], 
        'tank_energy': []
    }

    for t, w, e, p in zip(data['sim_time'], data['wrench_norm'], 
                          data['experiment_type'], data['power_from_interaction']):
        f = interp1d(t, w, kind='nearest')
        w = f(global_time)
        data_extended['time'].extend(global_time)
        data_extended['wrench_norm'].extend(w)
        label = ""
        if e == "no_filter":
            label = "without filter"
        elif e == "filter_in_out":
            label = "with filter"
        else:
            label = "with filter (zbf)"
        data_extended['experiment_type'].extend([label] * len(global_time))

        # we know that 10 is the initial energy value and we collect a sample each 15 steps of 1ms each
        ft = interp1d(t, 10 + np.cumsum(p)*0.015)
        s = ft(global_time)
        data_extended['tank_energy'].extend(s)

    data_extended = pd.DataFrame.from_dict(data_extended)

    fig, ax = plt.subplots()
    sns.lineplot(x='time', y='wrench_norm', hue="experiment_type", estimator='median', data=data_extended, ci='sd',
                 lw=3)

    fix_window = np.logical_and(global_time>t_min_fix, global_time<t_max_rel)
    ax.fill_between(global_time, 0, 1, where=fix_window, facecolor='red', alpha=0.1, transform=ax.get_xaxis_transform(), 
                    label="fix")
    ax.legend(fontsize=20)
    ax.set_xlim(5, 40)     
    ax.tick_params(axis='x', which='major', labelsize=20)
    ax.tick_params(axis='y', which='major', labelsize=20)
    ax.set_xlabel("time [s]", fontsize=20)
    ax.set_ylabel("wrench norm [N]", fontsize=20)

    fig.set_size_inches(10, 7)
    display_save_button(fig)



    fig, ax = plt.subplots()
    fig.set_size_inches(10, 7)

    rect = [0.5,0.4,0.48,0.48]
    ax.set_xlim(5, 40)
    sns.lineplot(x='time', y='tank_energy', hue="experiment_type", estimator='mean', data=data_extended, 
                 ci='sd', lw=3)
    ax.tick_params(axis='x', which='major', labelsize=20)
    ax.tick_params(axis='y', which='major', labelsize=20)
    ax.set_xlabel("time [s]", fontsize=20)
    ax.set_ylabel("energy [J]", fontsize=20)
    ax.legend(fontsize=20)

    ax_sub = add_subplot_axes(ax,rect)
    ax_sub.set_xlim(15, 40)
    ax_sub.set_ylim(-1, 11)
    sns.lineplot(x='time', y='tank_energy', hue="experiment_type", estimator='mean', data=data_extended, 
                 ci='sd', ax=ax_sub, legend=False, lw=3)
    ax_sub.axhline(y=2, xmin=0, xmax=1, ls="--", c='r', lw=3, label="min energy")
    ax_sub.legend(fontsize=20)
    ax_sub.set_xlabel("")
    ax_sub.set_ylabel("")
    ax_sub.tick_params(axis='x', which='major', labelsize=12)
    ax_sub.tick_params(axis='y', which='major', labelsize=12)
    display_save_button(fig)

<a id='passivity_coefficient_analysis'></a>

### Comparison energy tank with different alpha
[index](#index)

In [ ]:
if DO_PASSIVITY_COEFFICIENT_ANALYSIS:
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 7)
    ax.set_xlim(5, 40)
    ax.set_ylim(-1, 11)
    sns.lineplot(x='time', y='tank_energy', hue="experiment_type", estimator='mean', data=data_extended, 
                 ci='sd', ax=ax, legend=True, lw=3)
    ax.axhline(y=2, xmin=0, xmax=1, ls="--", c='r', lw=3, label="min energy")
    replace_legend_labels(ax, 
                          old_labels=["with filter", "with filter (zbf)"], 
                          new_labels=[r'with filter ($\alpha=1000.0$)', r'with filter ($\alpha=1.0$)'],
                          fontsize=20)

    ax.set_xlabel("time [s]", fontsize=20)
    ax.set_ylabel("energy [J]", fontsize=20)
    ax.tick_params(axis='x', which='major', labelsize=20)
    ax.tick_params(axis='y', which='major', labelsize=20)
    display_save_button(fig)

<a id='experiment_plots'></a>

## Experiment plots
[index](#index)

Here we plot different signal for a singal experiment

In [ ]:
exp_idx = 0    # choose the experiment

### Energy tank

In [ ]:
if DO_EXPERIMENT_PLOTS:
    fig, ax  = plt.subplots()
    ax.plot(data['sim_time'][exp_idx], 0.5 * np.square(data['tank_state'][exp_idx]), label="tank_energy")
    ax.plot(data['sim_time'][exp_idx], 10 + np.cumsum(data['power_from_interaction'][exp_idx])*0.015, 
            label="power_from_interaction")
    
    MIN_ENERGY = 2
    ax.axhline(y=MIN_ENERGY, ls="--")
    ax.set_title("Energy tank")
    ax.legend()

### Torque measurements

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = matrix_plot(data['sim_time'][exp_idx], 
                     data['external_torque'][exp_idx], 
                     prefix="external_torque")
    ax.set_title("External measured torque")
    ax.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)

    ax = matrix_plot(data['sim_time'][exp_idx], 
                     data['ground_truth_external_torque'][exp_idx], 
                     prefix="gt_external_torque")
    ax.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)

    # Compare estimated against ground truth
    uidx = 0
    fig, ax2 = plt.subplots()
    ax2.plot(data['sim_time'][exp_idx], data['external_torque'][exp_idx][:, uidx], label=f"est_{uidx}")
    ax2.plot(data['sim_time'][exp_idx], data['ground_truth_external_torque'][exp_idx][:, uidx], '--', 
             label=f"gt_{uidx}")
    ax2.legend()

### Stage cost

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = scalar_plot(data['sim_time'][exp_idx], data['stage_cost'][exp_idx], prefix="stage_cost")
    ax.set_title("stage_cost")

### Min cost

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = scalar_plot(data['sim_time'][exp_idx], data['min_cost'][exp_idx], prefix="min_cost")
    ax.set_title("Minimum rollout cost")

### Position desired

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = matrix_plot(data['sim_time'][exp_idx], data['position_desired'][exp_idx], prefix="des")
    ax = matrix_plot(data['sim_time'][exp_idx], data['position_measured'][exp_idx], prefix="meas", 
                     linestyle="--", axis=ax)
    ax.legend(fontsize=10)

    # Compare only one channel
    uidx = 2
    fig2, ax2 = plt.subplots()
    ax2.plot(data['sim_time'][exp_idx], data['position_desired'][exp_idx][:, uidx], label=f"desired_{uidx}")
    ax2.plot(data['sim_time'][exp_idx], data['position_measured'][exp_idx][:, uidx], '--', label=f"measured_{uidx}")
    ax2.legend()

### Velocity measured

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = matrix_plot(data['sim_time'][exp_idx], data['velocity_measured'][exp_idx], prefix="velocity_measured")
    ax.set_title("Velocity measured")

### Velocity command

In [ ]:
if DO_EXPERIMENT_PLOTS:
    cmd_idx = 6
    ax = scalar_plot(data['sim_time'][exp_idx], data['velocity_mppi'][exp_idx][:,cmd_idx], 
                     prefix=f"before_filter_{cmd_idx}")
    ax = scalar_plot(data['sim_time'][exp_idx], data['velocity_command'][exp_idx][:,cmd_idx], 
                    prefix=f"after_filter_{cmd_idx}", axis=ax)
    ax.set_title("Velocity command")

### Torque command

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = matrix_plot(data['sim_time'][exp_idx], data['torque_command'][exp_idx], prefix="torque_command")
    ax.set_title("Torque command")

### Joint limits violation

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = matrix_plot(data['sim_time'][exp_idx], data['joint_limits_violation'][exp_idx], 
                 prefix="joint_limits_violation")
    ax.set_title("Joint limits violation")

### Cartesian limits violation

In [ ]:
if DO_EXPERIMENT_PLOTS:
    ax = matrix_plot(data['sim_time'][exp_idx], data['cartesian_limits_violation'][exp_idx], 
            prefix="cartesian_limits_violation")
    ax.set_title("Cartesian limits violation")

### Computation Performance Metrics

In [ ]:
if DO_EXPERIMENT_PLOTS:
    #ax = scalar_plot(data['sim_time'][exp_idx], data['delay_steps'][exp_idx], prefix="delay_steps")
    #ax.set_title("delay steps")
    ax = scalar_plot(data['sim_time'][exp_idx], 1.0/data['rate'][exp_idx], prefix="rate")
    
    fig, ax = plt.subplots()
    contact_state_idx = 26
    contact_mask = data['state'][exp_idx][:, contact_state_idx] > 0.0
    sim_step = data['simulation_step'][exp_idx][contact_mask]
    ax.plot(data['state'][exp_idx][:, contact_state_idx] , label="contact flag")
    ax.plot(data['simulation_step'][exp_idx] * 1000 , label="simulation_step")

    # We compute these metrics only when in contact as this is where the difference between meshes plays 
    # a role
    print(f"Max: {1.0/np.min(sim_step)/1000}, Min: {1.0/np.max(sim_step)/1000} KHz, Mean: {1.0/np.mean(sim_step)/1000} KHz, Std: {1.0/np.std(sim_step)/1000} KHz")
    print(f"Average opt time: {np.mean(data['opt_time'][exp_idx]) * 1000 } ms")
    print("\n\nSingle")
    print("Max: 39.91856612510479, Min: 0.08300425604322863 KHz, Avg: 2.940748217192103 KHz, Std: 1.7962439317248262 KHz")
    print("Two realistic")
    print("Max: 41.94454930581771, Min: 0.04337963820514144 KHz, Avg: 2.1332271154104614 KHz, Std: 1.2765470097347171 KHz")
    print("Convex hull")
    print("Max: 32.96739524610161, Min: 0.04788936484000188 KHz, Avg: 0.9756731241637593 KHz, Std: 0.7493113313931056 KHz")